In [1]:
import onnx_graphsurgeon as gs
import onnx
import numpy as np

In [2]:
graph = gs.import_onnx(onnx.load("Swin_T.onnx"))

In [3]:
graph

[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _ipython_display_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_mimebundle_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_html_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_markdown_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_svg_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_png_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_pdf_ registered for opset: 11
[E] No function: _ipython_ca

Graph torch-jit-export (Opset: 11)
Inputs: [Variable (input): (shape=[1, 3, 224, 224], dtype=float32)]
Nodes:
Conv_0 (Conv)
	Inputs: [
		Variable (input): (shape=[1, 3, 224, 224], dtype=float32)
		Constant (patch_embed.proj.weight): (shape=[96, 3, 4, 4], dtype=<class 'numpy.float32'>)
		Constant (patch_embed.proj.bias): (shape=[96], dtype=<class 'numpy.float32'>)
	]
	Outputs: [
		Variable (191): (shape=[1, 96, 56, 56], dtype=float32)
	]
Attributes: OrderedDict([('dilations', [1, 1]), ('group', 1), ('kernel_shape', [4, 4]), ('pads', [0, 0, 0, 0]), ('strides', [4, 4])])
Shape_1 (Shape)
	Inputs: [
		Variable (191): (shape=[1, 96, 56, 56], dtype=float32)
	]
	Outputs: [
		Variable (192): (shape=[4], dtype=int64)
	]
Constant_2 (Constant)
	Inputs: [
	]
	Outputs: [
		Variable (193): (shape=[1], dtype=int64)
	]
Attributes: OrderedDict([('value', Constant (): (shape=[1], dtype=<class 'numpy.int64'>)
LazyValues (shape=[1], dtype=int64))])
Constant_3 (Constant)
	Inputs: [
	]
	Outputs: [
		Variable

In [4]:
def Add_plugin(graph):
    plugin = gs.Node(op = "WindowProcessPlugin_TRT")
    graph.nodes.append(plugin)
    wp_input_tensor = [node.outputs[0] for node in graph.nodes if node.name == "Reshape_105"][0]
    wp_output_tensor = [node.inputs[0] for node in graph.nodes if node.name == "Reshape_124"][0]
    wp_input_tensor.outputs[0] = plugin
    wp_output_tensor.inputs[0] = plugin

In [5]:
Add_plugin(graph)

In [3]:
def Add_reshape(graph):
    error_tensor = gs.Variable(name="error_tensor", dtype=np.int64, shape=(1, 10, 200))
    error_tensor0 = gs.Variable(name="error_tensor0", dtype=np.int64, shape=(1, 10, 200))
    for node in graph.nodes:
        if node.op == "OneHot1DPlugin_TRT":
            node.outputs = [error_tensor]
        if node.name == "Cast_147":
            node.inputs = [error_tensor0]
    reshape = gs.Node(op="Reshape", name="reshape_plugin")
    graph.nodes.append(reshape)
    shape = gs.Constant("reshape", np.array(error_tensor.shape))
    reshape.inputs = [error_tensor, shape]
    reshape.outputs = [error_tensor0]

In [4]:
Add_reshape(graph)

In [6]:
graph.cleanup().toposort()

[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _ipython_display_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_mimebundle_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_html_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_markdown_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_svg_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_png_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_pdf_ registered for opset: 11
[E] No function: _ipython_ca

Graph torch-jit-export (Opset: 11)
Inputs: [Variable (input): (shape=[1, 3, 224, 224], dtype=float32)]
Nodes:
Conv_0 (Conv)
	Inputs: [
		Variable (input): (shape=[1, 3, 224, 224], dtype=float32)
		Constant (patch_embed.proj.weight): (shape=[96, 3, 4, 4], dtype=<class 'numpy.float32'>)
		Constant (patch_embed.proj.bias): (shape=[96], dtype=<class 'numpy.float32'>)
	]
	Outputs: [
		Variable (191): (shape=[1, 96, 56, 56], dtype=float32)
	]
Attributes: OrderedDict([('dilations', [1, 1]), ('group', 1), ('kernel_shape', [4, 4]), ('pads', [0, 0, 0, 0]), ('strides', [4, 4])])
Constant_2 (Constant)
	Inputs: [
	]
	Outputs: [
		Variable (193): (shape=[1], dtype=int64)
	]
Attributes: OrderedDict([('value', Constant (): (shape=[1], dtype=<class 'numpy.int64'>)
LazyValues (shape=[1], dtype=int64))])
Constant_3 (Constant)
	Inputs: [
	]
	Outputs: [
		Variable (194): (shape=[1], dtype=int64)
	]
Attributes: OrderedDict([('value', Constant (): (shape=[1], dtype=<class 'numpy.int64'>)
LazyValues (shape=[1

In [7]:
onnx.save_model(gs.export_onnx(graph), "Swin_T_modified.onnx")

In [37]:
graph.nodes[458]

 (WindowProcessPlugin)
	Inputs: [
		Variable (373): (shape=None, dtype=None)
	]
	Outputs: [
		Variable (413): (shape=None, dtype=None)
	]

In [7]:
!python3 -m pip install colored

  Preparing metadata (setup.py) ... done
  Created wheel for colored: filename=colored-1.4.4-py3-none-any.whl size=14269 sha256=5c09ae4d026c09d0d15030239399773929b22c4f3daacfaf89ab685a5d3e6a7a
  Stored in directory: /root/.cache/pip/wheels/95/b9/be/9472bb50db786f327678fa8e575fce2c268593118a9257a722
Successfully built colored
